### Merging on different sets of fields

In [27]:
# Import pandas 
import pandas as pd

# Load the customer_data
customer_data = pd.read_csv('/Users/xingkaiwu/Library/CloudStorage/OneDrive-YorkUniversity/MBAN Winter/MBAN 6400/Customer Analytics and AB Testing in Python/user_demographics_v1.csv')
customer_data = customer_data.rename({'reg_date':'date'},axis = 1)
customer_data['date'] = pd.to_datetime(customer_data['date']).dt.date
# Load the app_purchases
app_purchases = pd.read_csv('/Users/xingkaiwu/Library/CloudStorage/OneDrive-YorkUniversity/MBAN Winter/MBAN 6400/Customer Analytics and AB Testing in Python/purchase_data_v1.csv')
app_purchases['date'] = pd.to_datetime(app_purchases['date']).dt.date
# Print the columns of customer data
print(customer_data.columns)

# Print the columns of app_purchases
print(app_purchases.columns)

Index(['uid', 'date', 'device', 'gender', 'country', 'age'], dtype='object')
Index(['date', 'uid', 'sku', 'price'], dtype='object')


In [29]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   uid      10000 non-null  float64
 1   date     10000 non-null  object 
 2   device   10000 non-null  object 
 3   gender   10000 non-null  object 
 4   country  10000 non-null  object 
 5   age      10000 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 468.9+ KB


In [30]:
# Merge on the 'uid' field
uid_combined_data = app_purchases.merge(customer_data, on=['uid'], how='inner')

# Examine the results 
print(uid_combined_data.head())
print(len(uid_combined_data))

       date_x       uid            sku  price      date_y device gender  \
0  2017-07-10  41195147  sku_three_499    499  2017-06-26    and      M   
1  2017-07-15  41195147  sku_three_499    499  2017-06-26    and      M   
2  2017-11-12  41195147   sku_four_599    599  2017-06-26    and      M   
3  2017-09-26  91591874    sku_two_299    299  2017-01-05    and      M   
4  2017-12-01  91591874   sku_four_599    599  2017-01-05    and      M   

  country  age  
0     BRA   17  
1     BRA   17  
2     BRA   17  
3     TUR   17  
4     TUR   17  
9006


In [31]:

# Merge on the 'uid' and 'date' field
uid_date_combined_data = app_purchases.merge(customer_data, on=['uid', 'date'], how='inner')

# Examine the results 
print(uid_date_combined_data.head())
print(len(uid_date_combined_data))

         date       uid             sku  price device gender country  age
0  2016-03-30  94055095    sku_four_599    599    iOS      F     BRA   16
1  2015-10-28  69627745     sku_one_199    199    and      F     BRA   18
2  2017-02-02  11604973  sku_seven_1499    499    and      F     USA   16
3  2016-06-05  22495315    sku_four_599    599    and      F     USA   19
4  2018-02-17  51365662     sku_two_299    299    iOS      M     TUR   16
35


### Practicing aggregations


In [32]:
# Calculate the mean purchase price 
purchase_price_mean = uid_combined_data.price.agg('mean')

# Examine the output 
print(purchase_price_mean)

406.77259604707973


In [33]:
# Calculate the mean and median purchase price 
purchase_price_summary = uid_combined_data.price.agg(['mean', 'median'])

# Examine the output 
print(purchase_price_summary)

mean      406.772596
median    299.000000
Name: price, dtype: float64


In [34]:
# Calculate the mean and median of price and age
purchase_summary = uid_combined_data.agg({'price': ['mean', 'median'], 'age': ['mean', 'median']})

# Examine the output 
print(purchase_summary)

             price        age
mean    406.772596  23.922274
median  299.000000  21.000000


### Grouping & aggregating


In [35]:
# Group the data 
grouped_purchase_data = uid_combined_data.groupby(by = ['device', 'gender'])

# Aggregate the data
purchase_summary = grouped_purchase_data.agg({'price': ['mean', 'median', 'std']})

# Examine the results
print(purchase_summary)

                    price                   
                     mean median         std
device gender                               
and    F       400.747504  299.0  179.984378
       M       416.237308  499.0  195.001520
iOS    F       404.435330  299.0  181.524952
       M       405.272401  299.0  196.843197


In [39]:
uid_combined_data.columns

Index(['date_x', 'uid', 'sku', 'price', 'date_y', 'device', 'gender',
       'country', 'age'],
      dtype='object')

### Calculating KPIs


In [40]:
from datetime import timedelta
current_date = pd.to_datetime('2018-03-17')

# Compute max_purchase_date 
max_purchase_date = current_date - timedelta(days=28)

# Filter to only include users who registered before our max date
purchase_data_filt = uid_combined_data[uid_combined_data.date_y < max_purchase_date]

# Filter to contain only purchases within the first 28 days of registration
purchase_data_filt = purchase_data_filt[(purchase_data_filt.date_x <= 
                        purchase_data_filt.date_y + timedelta(days=28))]

# Output the mean price paid per purchase
print(purchase_data_filt.price.mean())

414.4237288135593


/Users/xingkaiwu/miniforge3/envs/tf/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


### Average purchase price by cohort


In [42]:
import numpy as np
# Set the max registration date to be one month before today
max_reg_date = current_date - timedelta(days=28)

# Find the month 1 values
month1 = np.where((uid_combined_data.date_y < max_reg_date) &
                 (uid_combined_data.date_x < uid_combined_data.date_y + timedelta(days=28)),
                  uid_combined_data.price, 
                  np.NaN)
                 
# Update the value in the DataFrame
uid_combined_data['month1'] = month1

# Group the data by gender and device 
purchase_data_upd = uid_combined_data.groupby(by=['gender', 'device'], as_index=False) 

# Aggregate the month1 and price data 
purchase_summary = purchase_data_upd.agg(
                        {'month1': ['mean', 'median'],
                        'price': ['mean', 'median']})

# Examine the results 
print(purchase_summary)

  gender device      month1              price       
                       mean median        mean median
0      F    and  388.204545  299.0  400.747504  299.0
1      F    iOS  432.587786  499.0  404.435330  299.0
2      M    and  413.705882  399.0  416.237308  499.0
3      M    iOS  433.313725  499.0  405.272401  299.0
